In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
import os
from datasets import Dataset
from google.colab import drive
from datasets import load_dataset

drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
data_path = data_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/split_tokens_short/train_data_short.json'

In [ ]:
data = load_dataset('json', data_files=data_path, split='train')
data

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doc_number', 'split', 'tokens', 'labels', 'number of tokens'],
    num_rows: 15889
})

In [ ]:
data.features

{'doc_number': Value(dtype='int64', id=None),
 'split': Value(dtype='int64', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'number of tokens': Value(dtype='int64', id=None)}

In [ ]:
pd.DataFrame(data[:])[['tokens', 'labels']].iloc[0]

tokens    [Design, Thinking, for, innovation, reflexion,...
labels    [O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...
Name: 0, dtype: object

In [ ]:
import datasets


data = data.cast_column("labels", datasets.Sequence(datasets.ClassLabel(names=['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM'])))
data.features

Casting the dataset:   0%|          | 0/15889 [00:00<?, ? examples/s]

{'doc_number': Value(dtype='int64', id=None),
 'split': Value(dtype='int64', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None),
 'number of tokens': Value(dtype='int64', id=None)}

In [ ]:
pd.DataFrame(data[:])[['tokens', 'labels']].iloc[0]

tokens    [Design, Thinking, for, innovation, reflexion,...
labels    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ...
Name: 0, dtype: object

In [ ]:
data = data.remove_columns(["doc_number", "split", "number of tokens"])
data

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 15889
})

In [ ]:
labeled_rows_idx = []
for idx, ls in enumerate(data['labels']):
    for i in ls:
        if i != 0:
            labeled_rows_idx.append(idx)
            break


len(labeled_rows_idx)


1144

In [ ]:
small_dataset = data.select(labeled_rows_idx)

In [ ]:
data = small_dataset.train_test_split(test_size=0.2)
data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 915
    })
    test: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 229
    })
})

In [ ]:
tags = data['train'].features['labels'].feature
tags.names

['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM']

In [ ]:
tags_dict = {tag: 0 for tag in tags.names}
print(tags_dict)

{'O': 0, 'B-NAME_STUDENT': 0, 'I-NAME_STUDENT': 0, 'B-URL_PERSONAL': 0, 'B-ID_NUM': 0, 'B-EMAIL': 0, 'I-STREET_ADDRESS': 0, 'I-PHONE_NUM': 0, 'B-USERNAME': 0, 'B-PHONE_NUM': 0, 'B-STREET_ADDRESS': 0, 'I-URL_PERSONAL': 0, 'I-ID_NUM': 0}


In [ ]:
def count_tags(example):
    for tag in example['labels']:
        tag = tags.int2str(tag)
        tags_dict[tag] += 1

# count tags in the train set
data['train'].map(count_tags)
print('Train dataset tags:', tags_dict)

# reset dict and count the tags in the test set
tags_dict = {tag: 0 for tag in tags.names}
data['test'].map(count_tags)
print('Validation dataset tags:', tags_dict)


Map:   0%|          | 0/915 [00:00<?, ? examples/s]

Train dataset tags: {'O': 330026, 'B-NAME_STUDENT': 1097, 'I-NAME_STUDENT': 870, 'B-URL_PERSONAL': 97, 'B-ID_NUM': 59, 'B-EMAIL': 31, 'I-STREET_ADDRESS': 20, 'I-PHONE_NUM': 15, 'B-USERNAME': 4, 'B-PHONE_NUM': 6, 'B-STREET_ADDRESS': 2, 'I-URL_PERSONAL': 1, 'I-ID_NUM': 1}


Map:   0%|          | 0/229 [00:00<?, ? examples/s]

Validation dataset tags: {'O': 83986, 'B-NAME_STUDENT': 268, 'I-NAME_STUDENT': 226, 'B-URL_PERSONAL': 13, 'B-ID_NUM': 19, 'B-EMAIL': 8, 'I-STREET_ADDRESS': 0, 'I-PHONE_NUM': 0, 'B-USERNAME': 2, 'B-PHONE_NUM': 0, 'B-STREET_ADDRESS': 0, 'I-URL_PERSONAL': 0, 'I-ID_NUM': 0}


In [ ]:
tags = data['train'].features['labels'].feature

index2tag = {idx:tag for idx, tag in enumerate(tags.names)}
tag2index = {tag:idx for idx, tag in enumerate(tags.names)}
index2tag, tag2index

({0: 'O',
  1: 'B-NAME_STUDENT',
  2: 'I-NAME_STUDENT',
  3: 'B-URL_PERSONAL',
  4: 'B-ID_NUM',
  5: 'B-EMAIL',
  6: 'I-STREET_ADDRESS',
  7: 'I-PHONE_NUM',
  8: 'B-USERNAME',
  9: 'B-PHONE_NUM',
  10: 'B-STREET_ADDRESS',
  11: 'I-URL_PERSONAL',
  12: 'I-ID_NUM'},
 {'O': 0,
  'B-NAME_STUDENT': 1,
  'I-NAME_STUDENT': 2,
  'B-URL_PERSONAL': 3,
  'B-ID_NUM': 4,
  'B-EMAIL': 5,
  'I-STREET_ADDRESS': 6,
  'I-PHONE_NUM': 7,
  'B-USERNAME': 8,
  'B-PHONE_NUM': 9,
  'B-STREET_ADDRESS': 10,
  'I-URL_PERSONAL': 11,
  'I-ID_NUM': 12})

In [ ]:
def create_tag_names(batch):
    tag_name = {'ner_tags_str': [tags.int2str(idx) for idx in batch['labels']]}
    return tag_name

In [ ]:
data = data.map(create_tag_names)
data

Map:   0%|          | 0/915 [00:00<?, ? examples/s]

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'ner_tags_str'],
        num_rows: 915
    })
    test: Dataset({
        features: ['tokens', 'labels', 'ner_tags_str'],
        num_rows: 229
    })
})

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = data['train'][0]['tokens']  # these are 'pretokenized' so it needs to tokenize it further
inputs = tokenizer(inputs, is_split_into_words=True)  # we need to tell the tokenizer the data is split into words otherwize it will just consider it as one input
print(inputs.tokens())

['[CLS]', '▁of', '▁Products', '▁,', '▁although', '▁probably', '▁it', '▁could', '▁have', '▁required', '▁more', '▁time', '▁to', '▁complete', '▁.', '▁9', '▁.', '▁Nevertheless', '▁,', '▁I', '▁decided', '▁to', '▁apply', '▁Visual', 'isation', '▁again', '▁but', '▁using', '▁this', '▁“', '▁outcome', '▁first', '▁', '”', '▁approach', '▁in', '▁the', '▁next', '▁step', '▁of', '▁this', '▁work', '▁where', '▁we', '▁wanted', '▁to', '▁drill', '▁down', '▁into', '▁Customers', '▁,', '▁defining', '▁a', '▁set', '▁of', '▁lower', '▁level', '“', 'Sub', '▁-', '▁Customers', '▁', '”', '▁.', '▁I', '▁scheduled', '▁another', '▁“', '▁alignment', '▁session', '▁', '”', '▁and', '▁,', '▁instead', '▁of', '▁starting', '▁from', '▁a', '▁predetermined', '▁list', '▁of', '▁“', '▁sub', '▁-', '▁Customers', '▁', '”', '▁I', '▁decided', '▁to', '▁ask', '▁the', '▁stakeholders', '▁to', '▁list', '▁what', '▁was', '▁,', '▁in', '▁their', '▁view', '▁,', '▁the', '▁job', '▁to', '▁be', '▁done', '▁,', '▁writing', '▁them', '▁down', '▁into', '▁stat

In [ ]:
inputs = data['train'][0]['tokens']  # these are 'pretokenized' so it needs to tokenize it further
inputs = tokenizer(inputs, is_split_into_words=True)  # we need to tell the tokenizer the data is split into words otherwize it will just consider it as one input
print(inputs.tokens())

['[CLS]', '▁of', '▁Products', '▁,', '▁although', '▁probably', '▁it', '▁could', '▁have', '▁required', '▁more', '▁time', '▁to', '▁complete', '▁.', '▁9', '▁.', '▁Nevertheless', '▁,', '▁I', '▁decided', '▁to', '▁apply', '▁Visual', 'isation', '▁again', '▁but', '▁using', '▁this', '▁“', '▁outcome', '▁first', '▁', '”', '▁approach', '▁in', '▁the', '▁next', '▁step', '▁of', '▁this', '▁work', '▁where', '▁we', '▁wanted', '▁to', '▁drill', '▁down', '▁into', '▁Customers', '▁,', '▁defining', '▁a', '▁set', '▁of', '▁lower', '▁level', '“', 'Sub', '▁-', '▁Customers', '▁', '”', '▁.', '▁I', '▁scheduled', '▁another', '▁“', '▁alignment', '▁session', '▁', '”', '▁and', '▁,', '▁instead', '▁of', '▁starting', '▁from', '▁a', '▁predetermined', '▁list', '▁of', '▁“', '▁sub', '▁-', '▁Customers', '▁', '”', '▁I', '▁decided', '▁to', '▁ask', '▁the', '▁stakeholders', '▁to', '▁list', '▁what', '▁was', '▁,', '▁in', '▁their', '▁view', '▁,', '▁the', '▁job', '▁to', '▁be', '▁done', '▁,', '▁writing', '▁them', '▁down', '▁into', '▁stat

In [ ]:
inputs.word_ids()

[None,
 0,
 2,
 3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 24,
 26,
 27,
 28,
 29,
 30,
 32,
 34,
 36,
 38,
 38,
 40,
 42,
 44,
 46,
 48,
 49,
 51,
 52,
 52,
 54,
 56,
 58,
 60,
 62,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 84,
 85,
 87,
 89,
 91,
 93,
 95,
 97,
 97,
 97,
 98,
 99,
 100,
 100,
 101,
 103,
 105,
 107,
 109,
 110,
 112,
 113,
 113,
 115,
 116,
 118,
 120,
 122,
 124,
 126,
 128,
 130,
 132,
 134,
 135,
 136,
 137,
 138,
 138,
 140,
 142,
 144,
 146,
 148,
 150,
 152,
 154,
 156,
 158,
 159,
 161,
 163,
 165,
 166,
 168,
 170,
 172,
 174,
 176,
 177,
 179,
 181,
 183,
 185,
 187,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 200,
 201,
 203,
 205,
 207,
 208,
 210,
 211,
 213,
 215,
 217,
 219,
 221,
 223,
 225,
 227,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 236,
 237,
 237,
 239,
 241,
 243,
 245,
 247,
 248,
 249,
 251,
 252,
 254,
 256,
 258,
 258,
 259,
 261,
 263,
 265,
 267,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 27

In [ ]:
len(inputs.tokens()), len(data['train'][0]['tokens'])

(227, 358)

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)

        elif word_id is None:
            new_labels.append(-100)  # used in Pytorch to tell it to ignore the tag

        else:
            label = labels[word_id]
            if label%2 == 1:
                label = label + 1
            new_labels.append(label)

    return new_labels


In [ ]:
labels = data['train'][0]['labels']
word_ids = inputs.word_ids()

In [ ]:
align_labels_with_tokens(labels, word_ids)

[-100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 -100]

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'],
                                 truncation=True,
                                 padding=True,
                                 is_split_into_words=True,

                                 )

    all_labels = examples['labels']

    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = data.map(tokenize_and_align_labels, batched=True, remove_columns=data['train'].column_names)
tokenized_dataset

Map:   0%|          | 0/915 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/229 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 915
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 229
    })
})

Data collection

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [ ]:
batch = data_collator([tokenized_dataset['train'][i] for i in range(2)])
batch

{'input_ids': tensor([[   1,  265, 4950,  ...,    0,    0,    0],
        [   1,  263, 2983,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100]])}

Metrics

In [ ]:
!pip install seqeval
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e6d8607bd42354701dba339042a88a03e616aec0b3bd657ce2a61ea510188b36
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load('seqeval')

In [ ]:
ner_fetature = data['train'].features['labels']
ner_fetature

Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None)

In [ ]:
label_names = ner_fetature.feature.names
label_names

['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM']

In [ ]:
labels = data['train'][0]['labels']
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [ ]:
preditctions = labels.copy()
preditctions[2] = 'O'

metric.compute(predictions=[preditctions], references=[labels])

{'NAME_STUDENT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

In [ ]:
from transformers import AutoModelForTokenClassification
from transformers import set_seed

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,
                                                       num_labels=tags.num_classes,
                                                       # gradient_checkpointing=False,
                                                       id2label=index2tag,
                                                       label2id=tag2index,
                                                        )

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainerCallback
from transformers import TrainingArguments
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, get_scheduler
from transformers import DebertaForTokenClassification, DebertaTokenizer
import torch
import matplotlib.pyplot as plt
from transformers import set_seed



batch_size = 3
num_epochs = 20
logging_steps = 100

# Define TrainingArguments with model checkpointing and saving the best epoch
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/models/results',          # output directory
    save_strategy="epoch",                      # Disable automatic saving
    evaluation_strategy="epoch",                # evaluate every epoch
    load_best_model_at_end=True,                #load the best model when finished training
    metric_for_best_model = 'recall',
    logging_dir='/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/models/logs',     # directory for storing logs
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_steps=logging_steps,
    learning_rate=2e-5,
    weight_decay=0.01,
    disable_tqdm=False,
    save_total_limit=1,
)


# Update the Trainer initialization to include the callbacks
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)


In [ ]:
%%time
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.016300,0.005736,0.826783,0.945092,0.881988,0.998248
2,0.005400,0.010156,0.755650,0.890183,0.817418,0.997526
3,0.002900,0.005370,0.858824,0.971714,0.911788,0.998689
4,0.005200,0.004829,0.875758,0.961730,0.916733,0.998689
5,0.000700,0.005524,0.867061,0.976705,0.918623,0.998751
6,0.000300,0.005782,0.871492,0.981697,0.923318,0.998787
7,0.000500,0.005627,0.880597,0.981697,0.928403,0.998898
8,0.002300,0.005027,0.884731,0.983361,0.931442,0.998910
9,0.000300,0.006180,0.874074,0.981697,0.924765,0.998861
10,0.001900,0.005741,0.867647,0.981697,0.921155,0.998763


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: user 51min 4s, sys: 17min 6s, total: 1h 8min 11s
Wall time: 1h 9min 15s


TrainOutput(global_step=4880, training_loss=0.0057871974226026265, metrics={'train_runtime': 4153.2423, 'train_samples_per_second': 4.406, 'train_steps_per_second': 1.469, 'total_flos': 7644195736990560.0, 'train_loss': 0.0057871974226026265, 'epoch': 16.0})

In [ ]:
# After training, access the path of the best checkpoint like this
best_ckpt_path = trainer.state.best_model_checkpoint
best_ckpt_path

'/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/models/results/checkpoint-3355'

In [ ]:
test_data_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/split_tokens_short/test_data_short.json'
testdata = load_dataset('json', data_files=test_data_path, split='train')
testdata

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doc_number', 'split', 'tokens', 'number of tokens'],
    num_rows: 26
})

In [ ]:
testdata.column_names

['doc_number', 'split', 'tokens', 'number of tokens']

In [ ]:
# dataset needs to be tokenised
# there is nothing to be aligned
def tokenize(examples):
    tokenized_inputs = tokenizer(examples['tokens'],
                                 truncation=True,
                                 padding=True,
                                 is_split_into_words=True,
                                 )
    return tokenized_inputs

tokenized_testset = testdata.map(tokenize, batched=True, remove_columns=testdata.column_names)
tokenized_testset

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 26
})

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = best_ckpt_path
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)


token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'NAME_STUDENT',
  'score': 0.99628145,
  'word': 'Sylvain',
  'start': 10,
  'end': 18}]

In [ ]:
testdata['tokens'][0]

pred0 = token_classifier(testdata['tokens'][0])

In [ ]:
type(pred0)

list

In [ ]:

pred_labels_list = []

for doc in testdata['tokens']:
  pred_labels_list.append(token_classifier(doc))

predictions = pd.DataFrame({'pred_labels': pred_labels_list})

predictions.head()

,pred_labels
0,"[[], [], [], [], [], [], [{'entity_group': 'NA..."
1,"[[], [], [], [], [], [], [], [], [], [], [], [..."
2,"[[{'entity_group': 'NAME_STUDENT', 'score': 0...."
3,"[[], [], [], [], [], [], [], [], [], [], [], [..."
4,"[[{'entity_group': 'NAME_STUDENT', 'score': 0...."


In [ ]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pred_labels  26 non-null     object
dtypes: object(1)
memory usage: 336.0+ bytes


In [ ]:
testdata

Dataset({
    features: ['doc_number', 'split', 'tokens', 'number of tokens'],
    num_rows: 26
})

In [ ]:
predictions_dataset = testdata.add_column('pred_labels', predictions['pred_labels'])
predictions_dataset

Dataset({
    features: ['doc_number', 'split', 'tokens', 'number of tokens', 'pred_labels'],
    num_rows: 26
})

In [ ]:
pred_save_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/split_tokens_short/predictions_short.json'

predictions_dataset.to_json(pred_save_path, orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

165173

In [ ]:
def replace_pred_labels(pred_list):
  new_labels = []
  for label in pred_list:
    if label == []:
      new_labels.append('O')
    else:
      new_labels.append(label[0]['entity_group'])
  return new_labels

pl = predictions_dataset['pred_labels'][0]
# replace_pred_labels(pl)

In [ ]:
new_pred = []
for i in predictions_dataset['pred_labels']:

  new_pred.append(replace_pred_labels(i))

In [ ]:
new_pred_df = pd.DataFrame({'new_pred_labels': new_pred})

new_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   new_pred_labels  26 non-null     object
dtypes: object(1)
memory usage: 336.0+ bytes


In [ ]:
predictions_dataset = testdata.add_column('new_pred_labels', new_pred_df['new_pred_labels'])
predictions_dataset

Dataset({
    features: ['doc_number', 'split', 'tokens', 'number of tokens', 'new_pred_labels'],
    num_rows: 26
})

In [ ]:
pred_save_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/split_tokens_short/predictions_short.json'

predictions_dataset.to_json(pred_save_path, orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

107311

In [ ]:
predictions_dataset['new_pred_labels']

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'NAME_STUDENT',
  'NAME_STUDENT',
  'O',
  'NAME_STUDENT',
  'NAME_STUDENT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'NAME_STUDENT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ID_NUM',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ID_NUM',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'NAME_STUDENT',
  'NAME_STUDENT',
  'O',
  'NAME_STUDENT',
  'NAME_STUDENT',
  'O',
  'NAME_STUDENT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'NAME_STUDENT',
  'O',
  'NAME_STUDENT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ID_NUM',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'NAME_STUDENT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'NAME_S

In [ ]:
df = pd.read_json(pred_save_path, lines=True)
# df.head()

In [ ]:
# drop irrelevant cols
df = df.drop(columns=['split', 'number of tokens'])
# df

In [ ]:

# Find duplicates for the df[col_x]
duplicates = df.duplicated(subset=['doc_number'], keep=False)
# print(duplicates)
# Create a new DataFrame to store the merged rows
new_rows = []

# Iterate through each duplicate value and merge the rows
for value in df.loc[duplicates, 'doc_number'].unique():
    # Get rows with the same value
    group = df[df['doc_number'] == value]
    merged_row = group.iloc[0].copy()
    for col in df.columns:
      if col != 'doc_number':
        merged_row[col] = group[col].sum()
    new_rows.append(merged_row)

concat_pred_df = pd.DataFrame(new_rows)
concat_pred_df

# df.head()


,doc_number,tokens,new_pred_labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, NAME_STUDENT, NAME_STUDENT,..."
2,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[NAME_STUDENT, NAME_STUDENT, O, O, O, O, O, O,..."
4,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[NAME_STUDENT, O, O, O, NAME_STUDENT, NAME_STU..."
6,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[O, O, O, NAME_STUDENT, O, NAME_STUDENT, NAME_..."
9,56,"[Assignment, :, , Visualization, , Reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, NAME_STUD..."
14,86,"[Cheese, Startup, -, Learning, Launch, ​by, El...","[NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ..."
17,104,"[Storytelling, , The, Path, to, Innovation, \...","[NAME_STUDENT, O, O, NAME_STUDENT, O, NAME_STU..."
19,112,"[Reflection, –, Learning, Launch, \n\n, Franci...","[O, O, NAME_STUDENT, ID_NUM, O, NAME_STUDENT, ..."
21,123,"[Gandhi, Institute, of, Technology, and, Manag...","[NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ..."


In [ ]:
# Find rows with unique values for 'doc_number'
unique_rows = df.drop_duplicates(subset=['doc_number'], keep=False)

unique_rows


,doc_number,tokens,new_pred_labels
16,93,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,...","[NAME_STUDENT, NAME_STUDENT, O, O, O, O, NAME_..."


In [ ]:
# add unique row to the prediction concatenated dataset
# Concatenate the original DataFrame and the unique rows
concat_pred_df = pd.concat([concat_pred_df, unique_rows], ignore_index=True)

concat_pred_df

,doc_number,tokens,new_pred_labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, NAME_STUDENT, NAME_STUDENT,..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[NAME_STUDENT, NAME_STUDENT, O, O, O, O, O, O,..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[NAME_STUDENT, O, O, O, NAME_STUDENT, NAME_STU..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[O, O, O, NAME_STUDENT, O, NAME_STUDENT, NAME_..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, NAME_STUD..."
5,86,"[Cheese, Startup, -, Learning, Launch, ​by, El...","[NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ..."
6,104,"[Storytelling, , The, Path, to, Innovation, \...","[NAME_STUDENT, O, O, NAME_STUDENT, O, NAME_STU..."
7,112,"[Reflection, –, Learning, Launch, \n\n, Franci...","[O, O, NAME_STUDENT, ID_NUM, O, NAME_STUDENT, ..."
8,123,"[Gandhi, Institute, of, Technology, and, Manag...","[NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ..."
9,93,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,...","[NAME_STUDENT, NAME_STUDENT, O, O, O, O, NAME_..."


In [ ]:
concat_pred_df = concat_pred_df.sort_values(by='doc_number')
concat_pred_df

,doc_number,tokens,new_pred_labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, NAME_STUDENT, NAME_STUDENT,..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[NAME_STUDENT, NAME_STUDENT, O, O, O, O, O, O,..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[NAME_STUDENT, O, O, O, NAME_STUDENT, NAME_STU..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[O, O, O, NAME_STUDENT, O, NAME_STUDENT, NAME_..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, NAME_STUD..."
5,86,"[Cheese, Startup, -, Learning, Launch, ​by, El...","[NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ..."
9,93,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,...","[NAME_STUDENT, NAME_STUDENT, O, O, O, O, NAME_..."
6,104,"[Storytelling, , The, Path, to, Innovation, \...","[NAME_STUDENT, O, O, NAME_STUDENT, O, NAME_STU..."
7,112,"[Reflection, –, Learning, Launch, \n\n, Franci...","[O, O, NAME_STUDENT, ID_NUM, O, NAME_STUDENT, ..."
8,123,"[Gandhi, Institute, of, Technology, and, Manag...","[NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ..."


In [ ]:
concat_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   doc_number       10 non-null     int64 
 1   tokens           10 non-null     object
 2   new_pred_labels  10 non-null     object
dtypes: int64(1), object(2)
memory usage: 320.0+ bytes


In [ ]:
pred_save_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/predictions.json'

concat_pred_df.to_json(pred_save_path, orient='records', lines=True)

In [ ]:
concat_pred_df.new_pred_labels

0    [O, O, O, O, O, O, NAME_STUDENT, NAME_STUDENT,...
1    [NAME_STUDENT, NAME_STUDENT, O, O, O, O, O, O,...
2    [NAME_STUDENT, O, O, O, NAME_STUDENT, NAME_STU...
3    [O, O, O, NAME_STUDENT, O, NAME_STUDENT, NAME_...
4    [O, O, O, O, O, O, O, O, O, O, O, O, NAME_STUD...
5    [NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ...
9    [NAME_STUDENT, NAME_STUDENT, O, O, O, O, NAME_...
6    [NAME_STUDENT, O, O, NAME_STUDENT, O, NAME_STU...
7    [O, O, NAME_STUDENT, ID_NUM, O, NAME_STUDENT, ...
8    [NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ...
Name: new_pred_labels, dtype: object

In [ ]:
def add_prefix(labels):
    new_labels = []
    prev_label = None
    for label in labels:
        if label == 'O':
            new_labels.append('O')
            prev_label = 'O'
        else:
            if prev_label != label:
                new_labels.append('B-' + label)
                prev_label = label
            else:
                new_labels.append('I-' + label)
    return new_labels


# Add prefixes to labels
concat_pred_df['pred_labels'] = concat_pred_df['new_pred_labels'].apply(add_prefix)

concat_pred_df


,doc_number,tokens,new_pred_labels,pred_labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, NAME_STUDENT, NAME_STUDENT,...","[O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUD..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[NAME_STUDENT, NAME_STUDENT, O, O, O, O, O, O,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[NAME_STUDENT, O, O, O, NAME_STUDENT, NAME_STU...","[B-NAME_STUDENT, O, O, O, B-NAME_STUDENT, I-NA..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[O, O, O, NAME_STUDENT, O, NAME_STUDENT, NAME_...","[O, O, O, B-NAME_STUDENT, O, B-NAME_STUDENT, I..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, NAME_STUD...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
5,86,"[Cheese, Startup, -, Learning, Launch, ​by, El...","[NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ...","[B-NAME_STUDENT, I-NAME_STUDENT, O, B-NAME_STU..."
9,93,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,...","[NAME_STUDENT, NAME_STUDENT, O, O, O, O, NAME_...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, B..."
6,104,"[Storytelling, , The, Path, to, Innovation, \...","[NAME_STUDENT, O, O, NAME_STUDENT, O, NAME_STU...","[B-NAME_STUDENT, O, O, B-NAME_STUDENT, O, B-NA..."
7,112,"[Reflection, –, Learning, Launch, \n\n, Franci...","[O, O, NAME_STUDENT, ID_NUM, O, NAME_STUDENT, ...","[O, O, B-NAME_STUDENT, B-ID_NUM, O, B-NAME_STU..."
8,123,"[Gandhi, Institute, of, Technology, and, Manag...","[NAME_STUDENT, NAME_STUDENT, O, NAME_STUDENT, ...","[B-NAME_STUDENT, I-NAME_STUDENT, O, B-NAME_STU..."


In [ ]:
pred_save_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/predictions.json'

concat_pred_df.to_json(pred_save_path, orient='records', lines=True)

In [ ]:
import json

# Define the file path
file_path = '/content/drive/MyDrive/predictions.json'

# Read the file and get the content
with open(file_path, 'r') as file:
    # Read the entire file as a string
    file_content = file.read()

# Split the file content based on line breaks or a specific delimiter
# Depending on the format, adjust the splitting mechanism accordingly
# In this example, we assume each object is on a new line
json_objects = file_content.split('\n')

# Remove any empty lines
json_objects = [obj for obj in json_objects if obj.strip()]

# Convert each JSON string to a dictionary
json_dicts = [json.loads(obj) for obj in json_objects]

# Wrap the list of dictionaries in a list
wrapped_data = json_dicts

# Save the wrapped data back to the file
with open(file_path, 'w') as file:
    # Write the list as a single JSON array
    json.dump(wrapped_data, file, indent=4)

print("JSON objects have been wrapped in a list and saved back to the file.")


JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [ ]:
import json

# Specify the path to the JSON file
file_path = '/content/drive/MyDrive/predictions.json'



# Open the JSON file in read mode
with open(file_path, 'r') as file:
    # Load the JSON data from the file
    data = json.load(file)

# Now `data` contains the content of the JSON file as a Python dictionary (or list, depending on the JSON structure)
print(data)

# You can now work with the `data` variable as a Python dictionary or list


[{'doc_number': 7, 'tokens': ['Design', 'Thinking', 'for', 'innovation', 'reflexion', '-', 'Avril', '2021', '-', 'Nathalie', 'Sylla', '\n\n', 'Challenge', '&', 'selection', '\n\n', 'The', 'tool', 'I', 'use', 'to', 'help', 'all', 'stakeholders', 'finding', 'their', 'way', 'through', 'the', 'complexity', 'of', 'a', 'project', 'is', 'the', ' ', 'mind', 'map', '.', '\n\n', 'What', 'exactly', 'is', 'a', 'mind', 'map', '?', 'According', 'to', 'the', 'definition', 'of', 'Buzan', 'T.', 'and', 'Buzan', 'B.', '(', '1999', ',', 'Dessine', '-', 'moi', ' ', "l'intelligence", '.', 'Paris', ':', 'Les', 'Éditions', "d'Organisation", '.', ')', ',', 'the', 'mind', 'map', '(', 'or', 'heuristic', 'diagram', ')', 'is', 'a', 'graphic', ' ', 'representation', 'technique', 'that', 'follows', 'the', 'natural', 'functioning', 'of', 'the', 'mind', 'and', 'allows', 'the', 'brain', "'s", ' ', 'potential', 'to', 'be', 'released', '.', 'Cf', 'Annex1', '\n\n', 'This', 'tool', 'has', 'many', 'advantages', ':', '\n\n',

In [ ]:
import pandas as pd


# Read the JSON file into a pandas DataFrame
df = pd.read_json(file_path)
df.drop(columns = ['new_pred_labels'], inplace=True)

# Display the first few rows of the DataFrame
print(df)




   doc_number                                             tokens  \
0           7  [Design, Thinking, for, innovation, reflexion,...   
1          10  [Diego, Estrada, \n\n, Design, Thinking, Assig...   
2          16  [Reporting, process, \n\n, by, Gilberto, Gambo...   
3          20  [Design, Thinking, for, Innovation, \n\n, Sind...   
4          56  [Assignment, :,   , Visualization,  , Reflecti...   
5          86  [Cheese, Startup, -, Learning, Launch, ​by, El...   
6          93  [Silvia, Villalobos, \n\n, Challenge, :, \n\n,...   
7         104  [Storytelling,  , The, Path, to, Innovation, \...   
8         112  [Reflection, –, Learning, Launch, \n\n, Franci...   
9         123  [Gandhi, Institute, of, Technology, and, Manag...   

                                         pred_labels  
0  [O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUD...  
1  [B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...  
2  [B-NAME_STUDENT, O, O, O, B-NAME_STUDENT, I-NA...  
3  [O, O, O, B-NAME_STUDENT, O,

In [ ]:
import pandas as pd

def print_and_save_labels_not_equal_to_O(df, output_file='submission.csv'):
    # Initialize a list to store the output lines
    output_lines = []

    # Initialize a counter to keep track of the row_id
    counter = 0

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the document number
        document = row['doc_number']

        # Get the tokens and pred_labels
        tokens = row['tokens']
        pred_labels = row['pred_labels']

        # Iterate through pred_labels and their indices
        for token_index, label in enumerate(pred_labels):
            # Check if the label is not equal to 'O'
            if label != 'O':
                # Create the output line in the specified format
                output_line = f"{counter},{document},{token_index},{label}"

                # Print the output line
                print(output_line)

                # Add the output line to the list
                output_lines.append(output_line)

                # Increment the counter for the next row_id
                counter += 1

    header = "row_id,document,token,label"
    output_lines.insert(0, header)
    # Save the output lines to a CSV file
    with open(output_file, 'w') as f:
        f.write('\n'.join(output_lines))

    print(f"\nOutput saved to '{output_file}'.")

# Example usage
# Assuming `df` is your pandas DataFrame containing the data
# Call the function with the DataFrame and optionally specify the output file path
# print_and_save_labels_not_equal_to_O(df, 'output.csv')

print_and_save_labels_not_equal_to_O(df)


0,7,6,B-NAME_STUDENT
1,7,7,I-NAME_STUDENT
2,7,9,B-NAME_STUDENT
3,7,10,I-NAME_STUDENT
4,7,18,B-NAME_STUDENT
5,7,26,B-ID_NUM
6,7,41,B-ID_NUM
7,7,52,B-NAME_STUDENT
8,7,53,I-NAME_STUDENT
9,7,55,B-NAME_STUDENT
10,7,56,I-NAME_STUDENT
11,7,58,B-NAME_STUDENT
12,7,66,B-NAME_STUDENT
13,7,68,B-NAME_STUDENT
14,7,97,B-ID_NUM
15,7,107,B-NAME_STUDENT
16,7,119,B-NAME_STUDENT
17,7,128,B-ID_NUM
18,7,140,B-NAME_STUDENT
19,7,146,B-NAME_STUDENT
20,7,147,B-ID_NUM
21,7,156,B-NAME_STUDENT
22,7,159,B-ID_NUM
23,7,181,B-NAME_STUDENT
24,7,189,B-ID_NUM
25,7,195,B-NAME_STUDENT
26,7,204,B-NAME_STUDENT
27,7,206,B-ID_NUM
28,7,218,B-NAME_STUDENT
29,7,223,B-NAME_STUDENT
30,7,231,B-NAME_STUDENT
31,7,232,B-ID_NUM
32,7,253,B-ID_NUM
33,7,257,B-ID_NUM
34,7,263,B-ID_NUM
35,7,268,B-ID_NUM
36,7,276,B-ID_NUM
37,7,281,B-ID_NUM
38,7,284,B-ID_NUM
39,7,297,B-ID_NUM
40,7,308,B-NAME_STUDENT
41,7,324,B-NAME_STUDENT
42,7,343,B-ID_NUM
43,7,351,B-ID_NUM
44,7,356,B-NAME_STUDENT
45,7,372,B-NAME_STUDENT
46,7,382,B-NAME_STUDENT
47,7,421,B-ID_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
